## REDDIT


Vamos a extraer los datos de los post que contienen la palabra 'Python'

https://www.reddit.com/search/?q=python

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager # sustituye al archivo
from selenium.webdriver.chrome.options import Options # opciones de chrome
from selenium.webdriver.common.by import By # By es para buscar por tag, clase, id...
#Opciones de chrome
opciones = webdriver.ChromeOptions()
#opciones.add_argument('--start-maximized')
opciones.add_argument('window-size=775,1400')
opciones.add_argument('--disable-extensions')
opciones.add_argument('--disable-blink-features=AutomationControlled')
opciones.add_argument('--no-sandbox')
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])

# Deshabilita la ventana emergente de selección del buscador
opciones.add_argument("--no-default-browser-check")
opciones.add_argument("--disable-search-engine-choice-screen")

# Evitar mensajes emergentes automáticos de configuración
opciones.add_argument("--disable-infobars")

# guardar las cookies
# opciones.add_argument('user-data-dir=cookies')    # mantiene las cookies
opciones.add_argument('--incognito') # modo incognito

In [10]:
url = 'https://www.reddit.com/search/?q=python'
driver = webdriver.Chrome(options = opciones)   # abre una ventana de chrome
driver.get(url)


In [6]:
#Vamos a cerra las cookies como siempre...
#No las podemos localizar porque estan dentro de un shadow_root
#Padre de nuestro shadow_root
cookies = driver.find_element(By.TAG_NAME,'reddit-cookie-banner')

In [7]:
cookies

<selenium.webdriver.remote.webelement.WebElement (session="a71d2ecb69b9620398617500523220f1", element="f.A5870118060B17C7858DC47B432DDBE6.d.3FE0FEBAF4B171A69316116950504911.e.154")>

In [8]:
cookies.find_element(By.XPATH, '//*[@id="reject-nonessential-cookies-button"]/button').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="reject-nonessential-cookies-button"]/button"}
  (Session info: chrome=128.0.6613.137); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x55a5ce38b2da <unknown>
#1 0x55a5ce059200 <unknown>
#2 0x55a5ce0a8aa2 <unknown>
#3 0x55a5ce0a8d41 <unknown>
#4 0x55a5ce09d386 <unknown>
#5 0x55a5ce0cd8fd <unknown>
#6 0x55a5ce09d279 <unknown>
#7 0x55a5ce0cda9e <unknown>
#8 0x55a5ce0ebdfd <unknown>
#9 0x55a5ce0cd673 <unknown>
#10 0x55a5ce09b473 <unknown>
#11 0x55a5ce09c47e <unknown>
#12 0x55a5ce3520db <unknown>
#13 0x55a5ce356071 <unknown>
#14 0x55a5ce33e9d5 <unknown>
#15 0x55a5ce356bf2 <unknown>
#16 0x55a5ce323b6f <unknown>
#17 0x55a5ce37a248 <unknown>
#18 0x55a5ce37a417 <unknown>
#19 0x55a5ce38a0cc <unknown>
#20 0x7fa5c79776d7 start_thread
#21 0x7fa5c79fb60c __clone3


Porque está petando el código? y no encuentra el elemento?

**PRESENCIA DE SHADOWS**

Los **shadow** son sombras que ocultan parte del código para que no pueda ser encontrado y poner trabas. Así nuestro driver no puede identificar la ubicación del botón de las cookies y no puede realizar la acción de manera automática sin que tengamos que hacer algunos pasos extra.

In [11]:
#Buscamos el shadow que envuelve nuestro elemento
shadow = driver.find_element(By.TAG_NAME,'reddit-cookie-banner').shadow_root

Misión cumplida. Ahora vamos a hacer un scroll y nos vamos a guardar todos los enlaces a los que tengamos acceso y vamos a acceder a uno de ellos en una pestaña nueva.

In [12]:
#Ahora tenemos que usar el shadow como si fuere el driver
shadow.find_element(By.ID, 'reject-nonessential-cookies-button').click()
#Para ubicar dentro dentro de Shadows podemos usar CSS_SELECTOR o ID, otro romperán

#shadow.find_element(By.CSS_SELECTOR,'#reject-nonessential-cookies-button').text

In [13]:
#Bajamos hasta el final de la página
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # todo el scroll

In [14]:
# vuelvo al principio
driver.execute_script("window.scrollTo(0, 0);")

In [16]:
#Vamos a abrir el primer enlace en una ventana nueva
#Buscamos un a con el tributo data-testid = 'post-title'
enlaces = driver.find_elements(By.CSS_SELECTOR,'a[data-testid="post-title"]')
enlaces.__len__()

50

In [18]:
enlaces[0].get_attribute('href')

'https://www.reddit.com/r/interestingasfuck/comments/1fc2100/python_taking_a_dump/'

In [20]:
#Para el primer enlace sacamos el href
enlace_vid_16 = enlaces[15].get_attribute('href')

In [21]:
#Vamos a decirle a driver que abra esa url en una nueva pestaña
driver.execute_script(f"window.open('{enlace_vid_16}')")

Esto abrirá el enlace en una nueva pestaña. Cuidado porque únicamente hace eso, abrir la nueva pestaña. Para poder realizar acciones sobre esa nueva pestaña tenemos que comprobar las ventanas que tenemos abiertas. El código de la pestaña nos da igual, lo que nos interesa es la posición.

In [22]:
#Cambiamos a la nueva ventana
#vamos a ver cuantas handles tiene
driver.window_handles

['3C35669DC0AEFF6C6C008136C30C29CC', '9A934D82B39032AB933365F869DA8A33']

In [27]:
#Cambiamos a la nueva ventana
driver.switch_to.window(driver.window_handles[1])

In [28]:
#Cerramos la ventana
driver.close()

Aunque hayamos cerrado la pestaña que teníamos abierta eso no quiere decir que se puedda interaccionar sobre ella. Hay que seleccionar realmente esa pestaña para poder realizar cualquier acción sobre ella.

In [29]:
#Cambiamos a la ventana principal
driver.switch_to.window(driver.window_handles[0])

In [30]:
driver.quit()

## Ejercicio
- acceder a REDDIT
- quitar las cookies
- darle al boton de comunidades
- abrir la primera entrada en una ventana nueva

In [42]:
# import librerias
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager # sustituye al archivo
from selenium.webdriver.chrome.options import Options # opciones de chrome
from selenium.webdriver.common.by import By # By es para buscar por tag, clase, id...

In [ ]:
# Opciones comentadas porque falta actualizar, como las de arriba del notebook
# se tiene que desabilitar el selector de buscdor

# Opciones de chrome
# opciones = webdriver.ChromeOptions()
# opciones.add_argument('--start-maximized')
# opciones.add_argument('window-size=775,1400')
# opciones.add_argument('--disable-extensions')
# opciones.add_argument('--disable-blink-features=AutomationControlled')
# opciones.add_argument('--no-sandbox')
# opciones.add_experimental_option('useAutomationExtension', False)
# opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
# opciones.add_argument("--no-default-browser-check")
# opciones.add_argument("--disable-search-engine-choice-screen")
# opciones.add_argument("--disable-infobars")

# guardar las cookies
# opciones.add_argument('user-data-dir=cookies')    # mantiene las cookies
# opciones.add_argument('--incognito') # modo incognito

In [44]:
url = 'https://www.reddit.com/search/?q=python'
driver = webdriver.Chrome(options = opciones)   # abre una ventana de chrome

# cargar la web
driver.get(url)

time.sleep(1)

# raíz de la shadow
cookies_shadow = driver.find_element(By.TAG_NAME,'reddit-cookie-banner').shadow_root
time.sleep(1)

# click en botón de rechazar cookies
cookies_shadow.find_element(By.ID, 'reject-nonessential-cookies-button').click()
time.sleep(2)

# click en botón de comunidades
driver.find_element(By.ID, 'search-results-page-tab-communities').click()
time.sleep(1)

# guardar los elementos que contienen los enlaces
enlaces = driver.find_elements(By.CSS_SELECTOR,'a[class="absolute inset-0"]')
print(f'Hay {enlaces.__len__()} enlaces')

# extraer el href de alguno de los enlaces
entrada = enlaces[0].get_attribute('href')
print(f'Enlace al que voy a acceder: {entrada}')

# abrir enlace en nueva ventana
driver.execute_script(f"window.open('{entrada}')")
time.sleep(2)

# mover a ventana abierta
driver.switch_to.window(driver.window_handles[1])
time.sleep(2)

# cerrar la ventana
driver.close()
time.sleep(1)

# cerrar driver
driver.quit()

Hay 24 enlaces
Enlace al que voy a acceder: https://www.reddit.com/r/Python
